# Create and submits a mobotix job to the Waggle scheduler.

It is expected to monitors or trigger by an event (`scan_event`) to submit the jobs.

In [ ]:
import yaml

def create_job_file(directions, nodes, ip, username, password, south, filename="dynamic_scan_job.yaml"):
     """
    Creates a job file dynamically for the Waggle scheduler.

    Parameters:
        directions (list): List of directions for scanning (e.g., ["NEH", "NEB", "NEG"]).
        nodes (dict): Dictionary of node names and their statuses (e.g., {"W020": True}).
        ip (str): IP address of the camera (e.g., "camera-mobotix-thermal").
        username (str): Username for the camera (e.g., "admin").
        password (str): Password for the camera (e.g., "wagglesage").
        south (str): South parameter value (e.g., "22").
        filename (str): The name of the output YAML file.

    Returns:
        str: The name of the generated job file.
    """
    job = {
        "name": "mobotix-scan-direction",
        "plugins": [
            {
                "name": "mobotix-scan-direction",
                "pluginSpec": {
                    "image": "registry.sagecontinuum.org/bhupendraraut/mobotix-scan:0.24.8.20",
                    "args": [
                        "--ip",
                        ip,
                        "--mode",
                        "direction",
                        "-south",
                        south,
                        "-pt",
                        f"{','.join(directions)}",
                        "-u",
                        username,
                        "-p",
                        password
                    ]
                }
            }
        ],
        "nodes": nodes,
        "scienceRules": [
            'schedule("mobotix-scan-direction"): cronjob("mobotix-scan-direction", "* * * * *")'
        ],
        "successCriteria": []
    }

    with open(filename, "w") as file:
        yaml.dump(job, file, default_flow_style=False)
    print(f"Job file {filename} created successfully.")
    return filename


In [ ]:
directions = ["NEH", "NEB", "NEG", "EH", "EB", "EG", "SEH", "SEB", "SEG", "SH", "SB", "SG", "SWH", "SWB", "SWG"]
nodes = {"W021": True, "V032": True}
ip = "camera-mobotix-thermal"
username = "admin"
password = "meinsm"
south = "15"

# Create a job file
job_file = create_job_file(directions, nodes, ip, username, password, south)
print(f"Generated job file: {job_file}")


Job file dynamic_scan_job.yaml created successfully.
Generated job file: dynamic_scan_job.yaml


In [ ]:

def submit_job(filename):
    try:
        # Set SES environment variables)
        subprocess.run(["export", "SES_HOST=https://es.sagecontinuum.org"], check=True, shell=True)
        subprocess.run(["export", "SES_USER_TOKEN=your_user_token"], check=True, shell=True) # ask Sean
        
        # Create job
        result = subprocess.run(["sesctl", "create", "--file-path", filename], check=True, capture_output=True, text=True)
        print(f"Job creation response: {result.stdout}")
        
        # Get job_id from the response
        job_id = yaml.safe_load(result.stdout).get("job_id")
        if not job_id:
            raise ValueError("Job ID not found in the response.")
        
        # Submit job
        result = subprocess.run(["sesctl", "submit", "--job-id", job_id], check=True, capture_output=True, text=True)
        print(f"Job submission response: {result.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"Error during job submission: {e.stderr}")


In [14]:
submit_job("dynamic_scan_job.yaml")

export APPLICATIONINSIGHTS_CONFIGURATION_CONTENT="{}"
export APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL="1"
export CLICOLOR="1"
export CLICOLOR_FORCE="1"
export COMMAND_MODE="unix2003"
export CONDA_DEFAULT_ENV="sage"
export CONDA_EXE="/Users/bhupendra/anaconda3/bin/conda"
export CONDA_PREFIX="/Users/bhupendra/anaconda3/envs/sage"
export CONDA_PREFIX_1="/Users/bhupendra/anaconda3"
export CONDA_PREFIX_2="/Users/bhupendra/anaconda3/envs/data"
export CONDA_PREFIX_3="/Users/bhupendra/anaconda3"
export CONDA_PROMPT_MODIFIER="(sage) "
export CONDA_PYTHON_EXE="/Users/bhupendra/anaconda3/bin/python"
export CONDA_ROOT="/Users/bhupendra/anaconda3"
export CONDA_SHLVL="4"
export CPL_ZIP_ENCODING="UTF-8"
export ELECTRON_RUN_AS_NODE="1"
export FORCE_COLOR="1"
export GDAL_DATA="/Users/bhupendra/anaconda3/envs/data/share/gdal"
export GDAL_DRIVER_PATH="/Users/bhupendra/anaconda3/envs/data/lib/gdalplugins"
export GIT_PAGER="cat"
export GSETTINGS_SCHEMA_DIR="/Users/bhupendra/anaconda3/envs/sage/share/glib

FileNotFoundError: [Errno 2] No such file or directory: 'sesctl'